In [51]:
import pandas as pd
import numpy as np
import scipy.optimize as sco

In [52]:
df = pd.read_csv("../data/Training Data_Case 3.csv", index_col=0)
returns = df.pct_change().dropna()
# df.head(252*2).to_dict()

In [55]:
import pandas as pd
import numpy as np
from scipy.stats import norm

def calculate_cvar(returns, alpha):
    """Calculate CVaR at alpha level."""
    sorted_returns = sorted(returns)
    index = int(alpha * len(sorted_returns))
    cvar = sum(sorted_returns[:index]) / index
    return cvar

def markowitz_cvar_portfolio(df, alpha=0.05):
    """Calculate Markowitz portfolio allocation using CVaR."""
    returns = df.pct_change().dropna()

    # Calculate mean, variance, skewness, and kurtosis of returns
    mean_returns = returns.mean()
    cov_matrix = returns.cov()
    skewness = returns.skew()
    kurtosis = returns.kurtosis()

    # Set up optimization problem
    num_assets = len(df.columns)
    weights = np.ones(num_assets) / num_assets

    def portfolio_cvar(weights):
        portfolio_return = np.dot(weights, mean_returns)
        portfolio_volatility = np.sqrt(np.dot(weights.T, np.dot(cov_matrix, weights)))
        z = norm.ppf(alpha)
        portfolio_cvar = - portfolio_return + (z * portfolio_volatility * (1 + (skewness.dot(weights) / 6) * z - ((kurtosis.dot(weights) - 3) / 24) * (z ** 2)))
        return portfolio_cvar

    # Minimize CVaR
    from scipy.optimize import minimize
    constraints = ({'type': 'eq', 'fun': lambda x: np.sum(x) - 1})
    bounds = tuple((0,1) for i in range(num_assets))
    optimized_results = minimize(portfolio_cvar, weights, method='SLSQP', bounds=bounds, constraints=constraints)

    # Calculate Sharpe ratio
    portfolio_return = np.dot(optimized_results.x, mean_returns)
    portfolio_volatility = np.sqrt(np.dot(optimized_results.x.T, np.dot(cov_matrix, optimized_results.x)))
    sharpe_ratio = (252 ** 0.5) * (portfolio_return) / portfolio_volatility

    # Print results
    print("Weights:", optimized_results.x)
    print("Portfolio CVaR:", -optimized_results.fun)
    print("Sharpe Ratio:", sharpe_ratio)


In [56]:
markowitz_cvar_portfolio(df)

Weights: [5.83891136e-16 1.44859655e-16 0.00000000e+00 7.03000409e-16
 1.91474731e-17 1.00000000e+00 1.06858966e-15 5.57277018e-16
 1.77576409e-16 2.60661814e-16]
Portfolio CVaR: 0.05229372241283008
Sharpe Ratio: 0.5856181940011806


In [ ]:
start = 252*3

sharpe_ratios = []
for i in range(start,len(returns)):
    results = get_results(returns[i:i+1])
    sharpe_ratios.append()
